# Data Preparation

In [ ]:
## Load TEDS-D 2017 Data into a Pandas Data Frame
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import numpy as np
import os

## Specify WD
#wd = "/home/abf/BINF667013_Final_Project/"
wd = (
    "/Users/adam/Documents/BINF667013_BigDataAnalyticsHealthcare/" +
    "Final_Project/TEDS_Study"
)
os.chdir(wd)

teds=pd.read_csv("datasets/tedsd_puf_2017.csv")
teds

## Convert '-9' to NA
teds=teds.replace({-9:np.nan})

### Count the number of complete cases in this dataset

In [ ]:
teds.dropna().shape

### Count the number of cases with valid observations for at least 50% of available measures

In [ ]:
mask=teds.apply(lambda x: np.isnan(x)).apply(lambda x: sum(x) < 39, axis=1)
teds[mask].shape

### Count the number of cases with valid observations for at least 75% of available measures

In [ ]:
mask=teds.apply(lambda x: np.isnan(x)).apply(lambda x: sum(x) < 19, axis=1)
teds[mask].shape

### Count the number of cases with valid observations for at least 90% of available measures

In [ ]:
mask=teds.apply(lambda x: np.isnan(x)).apply(lambda x: sum(x) < 14, axis=1)
teds[mask].shape

### Explore Correlation between missing values

In [ ]:
import missingno as msno
msno.heatmap(teds, fontsize=12)

In [ ]:
msno.dendrogram(teds)

### Restrict data to records that are at least 75% complete

In [ ]:
mask=teds.apply(lambda x: np.isnan(x)).apply(lambda x: sum(x) < 19, axis=1)
print(teds[mask].shape)
teds = teds[mask]

## make a subset for development
#teds=teds.iloc[0:16300, ]

### Convert Observations to Categorical Variables

In [ ]:
for col in teds.columns:
    if not (col =="CASEID" or col == "DISYR"):
        teds[col] = teds[col].astype('category')

### Use MissForest implemented in missingpy to impute missing values

In [ ]:
## Stupid hack due to broken module name #WastingMyTime #SloppyCode
import sys
import sklearn.neighbors
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest
from os.path import exists

def random_chunks(df, n=100000, state=123):
    chunks = []
    i = 0
    while df.shape[0] > n:
        chunks.append(df.sample(n=n, replace=False, random_state=state))
        df = df.drop(chunks[i].index)
        i += 1

    chunks.append(df)
    return(chunks)
        
        
def impute_missforest(df, seed=123):
    print(df.index)
    imputer = MissForest(
        random_state=seed,
        criterion=('squared_error', 'gini'), # Package mispecifies criterion
        n_jobs=-1
    )
    X=imputer.fit_transform(
        df.drop(columns=['CASEID', 'DISYR']),
        cat_vars=np.array([x for x in range(0,74)])
    )

    dg=pd.DataFrame(X)
    dg.columns=df.columns[2:76]
    return(dg)

### Verfiy that All Chunks have at least 1000 observations for all variables

In [ ]:
chunks = random_chunks(df=teds, n=100000)
for c in chunks:
    x=c.shape[0] - c.apply(lambda x: sum(pd.isnull(x)), axis=0)
    print(c.index)
    print(x.loc[ x < 1000,])


In [ ]:
print(pd.concat(chunks).shape)

In [ ]:
if exists("analysis_objects/teds_imputed.csv"):
    teds_imputed=pd.read_csv(
        "analysis_objects/teds_imputed.csv",
        index_col=0
    )
else:
    chunks = random_chunks(df=teds, n=100000)
    dflist = [impute_missforest(c) for c in chunks]
    teds_imputed = pd.concat(dflist)
    teds_imputed.to_csv(
        path_or_buf="analysis_objects/teds_imputed.csv"
    )

### Prepare State Level Data

In [ ]:
## Import Involuntary Commitment Laws dataset
ic_laws=pd.read_csv("datasets/IC_Laws.csv")

## Convert "." character to NA in appropriate columns
ic_laws=ic_laws.replace({".":np.nan})
## Add state identifier that matches TEDS data (as join column). 
fips_table=pd.read_csv('datasets/state.txt', sep="|")
ic_laws = pd.merge(
    left=fips_table.loc[:,["STATE", "STATE_NAME"]],
    right=ic_laws,
    left_on="STATE_NAME",
    right_on="Jurisdiction",
    how='inner'
).drop(columns="STATE_NAME")
ic_laws['pre_2016'] = ic_laws['Effective Date'] < "2016-01-01"
ic_laws.STATE.value_counts()


### Join State Laws to TEDS Data

In [ ]:
if exists("analysis_objects/teds_laws.csv"):
    teds_laws=pd.read_csv("analysis_objects/teds_laws.csv")
else:
    teds_laws = pd.merge(
        left=teds,
        right=ic_laws,
        left_on="STFIPS",
        right_on="STATE",
    )
    teds_laws.to_csv(
        "analysis_objects/teds_laws.csv", index=False
    )
    
if exists("analysis_objects/teds_imp_laws.csv"):
    teds_imp_laws=pd.read_csv("analysis_objects/teds_imp_laws.csv")
else:
    teds_imp_laws=pd.merge(
        left=teds_imputed,
        right=ic_laws,
        left_on="STFIPS",
        right_on="STATE",
    )
    teds_imp_laws.to_csv(
        "analysis_objects/teds_imp_laws.csv", index=False
    )

# Exploratory Data Analysis

### Summarize Relapses by State And Drug
Cases where the primary substance at admission is still being used at discharge will be considered relapses


In [ ]:
### Define A function to flag relapses
def relapse(x, **kwargs):
    if  (
            x['SUB1'] == kwargs['drug'] and (
            x['SUB1_D'] == kwargs['drug'] or\
            x['SUB2_D'] == kwargs['drug'] or\
            x['SUB3_D'] == kwargs['drug']
        )
    ):
        return 1
    else:
        return 0

### Flag Relapses in data
#test = teds_laws.copy()
#test.loc[:,'MET_RELA'] = test.loc[:,['CASEID','SUB1', 'SUB1_D', 'SUB2_D', 'SUB3_D']].apply(relapse, axis=1, drug=5)


In [ ]:
## Calculate Number of Relapses Per State
teds_imp_laws = teds_imp_laws.assign(
    alc_cases = lambda x: x.loc[:,['SUB1']].apply(lambda x: x['SUB1'] == 2, axis=1),
    hrn_cases = lambda x: x.loc[:,['SUB1']].apply(lambda x: x['SUB1'] == 5, axis=1),
    met_cases = lambda x: x.loc[:,['SUB1']].apply(lambda x: x['SUB1'] == 10, axis=1),
    alc_relapse = lambda x: x.loc[:,['SUB1', 'SUB1_D', 'SUB2_D', 'SUB3_D']].apply(relapse, axis=1, drug=2),
    hrn_relapse = lambda x: x.loc[:,['SUB1', 'SUB1_D', 'SUB2_D', 'SUB3_D']].apply(relapse, axis=1, drug=5),
    met_relapse = lambda x: x.loc[:,['SUB1', 'SUB1_D', 'SUB2_D', 'SUB3_D']].apply(relapse, axis=1, drug=10)
)
teds_imp_laws.groupby(['Jurisdiction']).sum().loc[:,['alc_cases','hrn_cases', 'met_cases','alc_relapse','hrn_relapse', 'met_relapse']].assign(
    alc_rate = lambda x: x['alc_relapse'] / x['alc_cases'],
    hrn_rate = lambda x: x['hrn_relapse'] / x['hrn_cases'],
    met_rate = lambda x: x['met_relapse'] / x['met_cases']
)

In [ ]:
## Index casese of alcohol use at intake
alc_cases = teds_imp_laws.alc_cases == 1

plt.figure(figsize=(6, 6))
plot_data = teds_imp_laws.loc[
    alc_cases,['IC_law','alc_cases','alc_relapse']
].groupby(['IC_law']).sum().reset_index().assign(
    relapse_rate = lambda x: 100*(x['alc_relapse'] / x['alc_cases']),
    total = [100, 100]
)
print(plot_data)
total_bars = sns.barplot(
    x="IC_law",  
    y="total",
    data=plot_data, 
    color='darkblue'
)

rate_bars = sns.barplot(
    x="IC_law",  
    y="relapse_rate",
    data=plot_data, 
    color='lightblue'
)
plt.show()




In [ ]:
## Index casese of alcohol use at intake
alc_cases = teds_imp_laws.alc_cases == 1

plt.figure(figsize=(6, 6))
plot_data = teds_imp_laws.loc[
    alc_cases,['IC_law','alc_cases','alc_relapse']
].groupby(['IC_law']).sum().reset_index().assign(
    relapse_rate = lambda x: 100*(x['alc_relapse'] / x['alc_cases']),
    total = [100, 100]
)
print(plot_data)
total_bars = sns.barplot(
    x="IC_law",  
    y="total",
    data=plot_data, 
    color='darkblue'
)

rate_bars = sns.barplot(
    x="IC_law",  
    y="relapse_rate",
    data=plot_data, 
    color='lightblue'
)
plt.show()




In [ ]:
## Index casese of alcohol use at intake
hrn_cases = teds_imp_laws.hrn_cases == 1
plt.figure(figsize=(6, 6))
plot_data = teds_imp_laws.loc[
    hrn_cases,['IC_law','hrn_cases','hrn_relapse']
].groupby(['IC_law']).sum().reset_index().assign(
    relapse_rate = lambda x: 100*(x['hrn_relapse'] / x['hrn_cases']),
    total = [100, 100]
)
print(plot_data)
total_bars = sns.barplot(
    x="IC_law",  
    y="total",
    data=plot_data, 
    color='darkblue'
)

rate_bars = sns.barplot(
    x="IC_law",  
    y="relapse_rate",
    data=plot_data, 
    color='lightblue'
)
plt.show()




In [ ]:
## Index casese of alcohol use at intake
met_cases = teds_imp_laws.met_cases == 1

plt.figure(figsize=(6, 6))
plot_data = teds_imp_laws.loc[
    met_cases,['IC_law','met_cases','met_relapse']
].groupby(['IC_law']).sum().reset_index().assign(
    relapse_rate = lambda x: 100*(x['met_relapse'] / x['met_cases']),
    total = [100, 100]
)
print(plot_data)
total_bars = sns.barplot(
    x="IC_law",  
    y="total",
    data=plot_data, 
    color='darkblue'
)

rate_bars = sns.barplot(
    x="IC_law",  
    y="relapse_rate",
    data=plot_data, 
    color='lightblue'
)
plt.show()




In [ ]:
## Calculate Refferal Rates by Category
totals=teds_imp_laws.loc[:,['IC_law', 'pre_2016', 'PSOURCE']].assign(
    ref_individual = lambda x: x.loc[:,['PSOURCE']].apply(lambda x: x['PSOURCE'] == 1, axis=1),
    ref_care = lambda x: x.loc[:,['PSOURCE']].apply(lambda x: x['PSOURCE'] == 2, axis=1),
    ref_health = lambda x: x.loc[:,['PSOURCE']].apply(lambda x: x['PSOURCE'] == 3, axis=1),
    ref_school = lambda x: x.loc[:,['PSOURCE']].apply(lambda x: x['PSOURCE'] == 4, axis=1),
    ref_employer = lambda x: x.loc[:,['PSOURCE']].apply(lambda x: x['PSOURCE'] == 5, axis=1),
    ref_community = lambda x: x.loc[:,['PSOURCE']].apply(lambda x: x['PSOURCE'] == 6, axis=1),
    ref_adjudicated = lambda x: x.loc[:,['PSOURCE']].apply(lambda x: x['PSOURCE'] == 7, axis=1)
).groupby(['IC_law', 'pre_2016']).sum().drop(columns=['PSOURCE'])

totals.div(totals.sum(axis=1), axis=0)



### Impact of Pre 2016 IC Laws on Length of Stay

In [ ]:
## Calculate the proportion of cases in IC_Law states for each level of
## length of stay
totals=teds_imp_laws.loc[:,['IC_law', 'pre_2016', 'LOS']].groupby(['LOS']).sum()
totals.div(teds_imp_laws.loc[:,['IC_law', 'pre_2016', 'LOS']].groupby(['LOS']).count(), axis=1)

In [ ]:
## Evaluate median Length of stay by IC_Law status
#teds_imp_laws.iloc[:,['IC_law']]
teds_imp_laws['IC_law']=teds_imp_laws['IC_law'].astype(
    pd.CategoricalDtype(
        categories =[0, 1], ordered=False)
)

teds_imp_laws['LOS']=teds_imp_laws['LOS'].astype('int')
bad_drugs=teds_imp_laws.SUB1.isin([2,5,10])
sns.catplot(
    x="IC_law", y="LOS", hue="SUB1", 
    kind="violin", data=teds_imp_laws.loc[bad_drugs,:]
)

In [ ]:

teds_imp_laws['LOS']=teds_imp_laws['LOS'].astype('int')
bad_drugs=teds_imp_laws.SUB1.isin([2,5,10])
sns.catplot(
    x="pre_2016", y="LOS", hue="SUB1",
    kind="violin", data=teds_imp_laws.loc[bad_drugs,:]
)

In [ ]:

teds_imp_laws['LOS']=teds_imp_laws['LOS'].astype('int')
bad_drugs=teds_imp_laws.SUB1.isin([2,5,10])
sns.catplot(
    x="pre_2016", y="LOS", hue="SUB1", row="SERVICES",
    kind="violin", data=teds_imp_laws.loc[bad_drugs,:]
)

### Distibution of SERVICES by IC Law Status

In [ ]:
## Index casese of alcohol use at intake
alc_cases = teds_imp_laws.alc_cases == 1

plt.figure(figsize=(6, 6))
plot_data = teds_imp_laws.loc[
    alc_cases,['IC_law','alc_cases','alc_relapse']
].groupby(['IC_law']).sum().reset_index().assign(
    relapse_rate = lambda x: 100*(x['alc_relapse'] / x['alc_cases']),
    total = [100, 100]
)
print(plot_data)
total_bars = sns.barplot(
    x="IC_law",  
    y="total",
    data=plot_data, 
    color='darkblue'
)

rate_bars = sns.barplot(
    x="IC_law",  
    y="relapse_rate",
    data=plot_data, 
    color='lightblue'
)
plt.show()


